In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import join
from PIL import Image
from sklearn.model_selection import train_test_split
import keras
import pandas as pd
import glob


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


Num GPUs Available:  1


In [3]:
import tensorflow as tf
from tensorflow import keras
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


### Clases:
- 0 : Normal.
- 1 : Falla en la pista interna.
- 2 : Falla pista externa.
- 3 : Falla en bola rodamiento.
- 4 : Falla combinada.

In [4]:
#Crear una función que divida en datos de entrenamiento y test las imagenes.

## Lectura de imágenes como arrays y creación de targets en base a etiquetas.

In [5]:
def prepros_bancoimagenes(mydir, test_size, val_size):
    #Lectura en array de imagenes, preprocesamiento a grises y reescalamiento a 128x128
    X = [] 
    for name in listdir(mydir) :
        path = join( mydir,name) 
        data = Image.open(path).convert('L')
        data = data.resize((128,128))
        data = np.array(data)
        X.append(data) 
    X = np.array(X) 
    print(X.shape) 

    y =[]
    for name in listdir(mydir) :
        path = join( mydir,name) 
        if name[0] == 'H':
            y.append(0)
        if name[0] == 'I':
            y.append(1)
        if name[0] == 'O':
            y.append(2)
        if name[0] == 'B':
            y.append(3)
        if name[0] == 'C':
            y.append(4)
    y = np.array(y) 
    print(y.shape) 
    
    # Dividir datos de test y entrenamiento.
    x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify = y)
    
    #Dividir los datos de train en entrenamiento y validación.
    x_train,x_val,y_train,y_val = train_test_split(x_train, y_train, test_size=val_size, random_state=42, stratify = y_train)
    
    #Normalización
    x_train = x_train.astype("float32") / 255
    x_val= x_val.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    
    #Agregar dimensión extra
    x_train = np.expand_dims(x_train, -1) 
    x_val = np.expand_dims(x_val, -1) 
    x_test = np.expand_dims(x_test, -1) 
    
    #One hot encoder
    y_train = keras.utils.to_categorical(y_train, 5)
    y_val = keras.utils.to_categorical(y_val, 5)
    y_test = keras.utils.to_categorical(y_test, 5)

    return x_train, x_test, x_val, y_train, y_test, y_val

In [43]:
#Directorios de bancos de imagenes

mydir_1 = r"BANCOS/grey/16384/1seg/Banco_GLOBAL_COLOR"
mydir_2 = r"BANCOS/grey/16384/1seg/Banco_LOCAL_COLOR" 
mydir_3 = r"BANCOS/grey/16384/1seg/Banco_SENAL_COLOR" 

mydir_4 = r"BANCOS/grey/16384/10giros/Banco_GLOBAL_COLOR"
mydir_5 = r"BANCOS/grey/16384/10giros/Banco_LOCAL_COLOR" 
mydir_6 = r"BANCOS/grey/16384/10giros/Banco_SENAL_COLOR" 

mydir_7 = r"BANCOS/grey/16384/0,6seg/Banco_GLOBAL_COLOR"
mydir_8 = r"BANCOS/grey/16384/0,6seg/Banco_LOCAL_COLOR" 
mydir_9 = r"BANCOS/grey/16384/0,6seg/Banco_SENAL_COLOR" 

test_size = 0.2
val_size = 0.1

In [44]:
x_train_1.shape

(424, 128, 128, 1)

In [45]:
#Datos
x_train_1, x_test_1, x_val_1, y_train_1, y_test_1, y_val_1 = prepros_bancoimagenes(mydir_1, test_size, val_size)
x_train_2, x_test_2, x_val_2, y_train_2, y_test_2, y_val_2 = prepros_bancoimagenes(mydir_2, test_size, val_size)
x_train_3, x_test_3, x_val_3, y_train_3, y_test_3, y_val_3 = prepros_bancoimagenes(mydir_3, test_size, val_size)

x_train_4, x_test_4, x_val_4, y_train_4, y_test_4, y_val_4 = prepros_bancoimagenes(mydir_4, test_size, val_size)
x_train_5, x_test_5, x_val_5, y_train_5, y_test_5, y_val_5 = prepros_bancoimagenes(mydir_5, test_size, val_size)
x_train_6, x_test_6, x_val_6, y_train_6, y_test_6, y_val_6 = prepros_bancoimagenes(mydir_6, test_size, val_size)

x_train_7, x_test_7, x_val_7, y_train_7, y_test_7, y_val_7 = prepros_bancoimagenes(mydir_7, test_size, val_size)
x_train_8, x_test_8, x_val_8, y_train_8, y_test_8, y_val_8 = prepros_bancoimagenes(mydir_8, test_size, val_size)
x_train_9, x_test_9, x_val_9, y_train_9, y_test_9, y_val_9 = prepros_bancoimagenes(mydir_9, test_size, val_size)

(590, 128, 128)
(590,)
(590, 128, 128)
(590,)
(590, 128, 128)
(590,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(944, 128, 128)
(944,)
(944, 128, 128)
(944,)
(944, 128, 128)
(944,)


### Creación del modelo - ARQUITECTURA FIJA

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D, Dropout


In [7]:
#Función que guardar historial, modelo y separa por nombres.
# ramdom state
def arquitectura_fija(x_train, y_train, x_val, y_val, name, epochs, batch,iteraciones):
    tf.random.set_seed(iteraciones)
    model = Sequential([
        Conv2D(64, (5,5), activation='relu', input_shape=(128, 128, 1)),
        MaxPooling2D(pool_size=(2,2), padding='same'),
        Conv2D(32, (5,5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2,2), padding='same'),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.4),
        Dense(5, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    hist = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch, verbose = 0)
    
    os.makedirs('resultados-cnn/models', exist_ok=True)
    model.save(os.path.join('resultados-cnn/models', name + '.h5'))
    
    os.makedirs('resultados-cnn/history', exist_ok=True)
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(os.path.join('resultados-cnn/history', name + '.csv'))



## Entrenamiento 10 vecs con CNN de arquitectura fija de LeNet

In [8]:
epochs = 10
batch = 32

In [9]:
def iterar_cnn(iteraciones, x_train, y_train, x_val, y_val,name,epochs,batch):
    for i in range(iteraciones):
        iter_name = name + '_iter_' + str(i)
        arquitectura_fija(x_train, y_train, x_val, y_val, iter_name, epochs, batch,iteraciones)

In [50]:
iteraciones = 10
iterar_cnn(iteraciones, x_train_1, y_train_1, x_val_1, y_val_1, 'GLOBAL-1seg-16384', epochs, batch)

In [51]:
iterar_cnn(iteraciones, x_train_2, y_train_2, x_val_2, y_val_2, 'LOCAL-1seg-16384', epochs, batch)

In [52]:
iterar_cnn(iteraciones, x_train_3, y_train_3, x_val_3, y_val_3, 'SENAL-1seg-16384', epochs, batch)

In [53]:
iterar_cnn(iteraciones, x_train_4, y_train_4, x_val_4, y_val_4, 'GLOBAL-10giros-16384', epochs, batch)

In [54]:
iterar_cnn(iteraciones, x_train_5, y_train_5, x_val_5, y_val_5, 'LOCAL-10giros-16384', epochs, batch)

In [55]:
iterar_cnn(iteraciones, x_train_6, y_train_6, x_val_6, y_val_6, 'SENAL-10giros-16384', epochs, batch)

In [56]:
iterar_cnn(iteraciones, x_train_7, y_train_7, x_val_7, y_val_7, 'GLOBAL-0,6seg-16384', epochs, batch)

In [57]:
iterar_cnn(iteraciones, x_train_8, y_train_8, x_val_8, y_val_8, 'LOCAL-0,6seg-16384', epochs, batch)

In [58]:
iterar_cnn(iteraciones, x_train_9, y_train_9, x_val_9, y_val_9, 'SENAL-0,6seg-16384', epochs, batch)

# Para resultados 2: Distintos anchos de ventana duración señal 10 giros.

In [10]:
#Directorios de bancos de imagenes

mydir_best_1= r"BANCOS/grey/4096/10giros/Banco_GLOBAL_COLOR"
mydir_best_2= r"BANCOS/grey/4096/10giros/Banco_LOCAL_COLOR" 
mydir_best_3= r"BANCOS/grey/4096/10giros/Banco_SENAL_COLOR" 

mydir_best_4= r"BANCOS/grey/8192/10giros/Banco_GLOBAL_COLOR"
mydir_best_5= r"BANCOS/grey/8192/10giros/Banco_LOCAL_COLOR" 
mydir_best_6= r"BANCOS/grey/8192/10giros/Banco_SENAL_COLOR" 

mydir_best_7= r"BANCOS/grey/16384/10giros/Banco_GLOBAL_COLOR"
mydir_best_8= r"BANCOS/grey/16384/10giros/Banco_LOCAL_COLOR" 
mydir_best_9= r"BANCOS/grey/16384/10giros/Banco_SENAL_COLOR" 

mydir_best_10 = r"BANCOS/grey/32768/10giros/Banco_GLOBAL_COLOR"
mydir_best_11 = r"BANCOS/grey/32768/10giros/Banco_LOCAL_COLOR"
mydir_best_12 = r"BANCOS/grey/32768/10giros/Banco_SENAL_COLOR" 

test_size = 0.2
val_size = 0.1

In [11]:
#Datos
x_train_1, x_test_1, x_val_1, y_train_1, y_test_1, y_val_1 = prepros_bancoimagenes(mydir_best_1, test_size, val_size)
x_train_2, x_test_2, x_val_2, y_train_2, y_test_2, y_val_2 = prepros_bancoimagenes(mydir_best_2, test_size, val_size)
x_train_3, x_test_3, x_val_3, y_train_3, y_test_3, y_val_3 = prepros_bancoimagenes(mydir_best_3, test_size, val_size)

x_train_4, x_test_4, x_val_4, y_train_4, y_test_4, y_val_4 = prepros_bancoimagenes(mydir_best_4, test_size, val_size)
x_train_5, x_test_5, x_val_5, y_train_5, y_test_5, y_val_5 = prepros_bancoimagenes(mydir_best_5, test_size, val_size)
x_train_6, x_test_6, x_val_6, y_train_6, y_test_6, y_val_6 = prepros_bancoimagenes(mydir_best_6, test_size, val_size)

x_train_7, x_test_7, x_val_7, y_train_7, y_test_7, y_val_7 = prepros_bancoimagenes(mydir_best_7, test_size, val_size)
x_train_8, x_test_8, x_val_8, y_train_8, y_test_8, y_val_8 = prepros_bancoimagenes(mydir_best_8, test_size, val_size)
x_train_9, x_test_9, x_val_9, y_train_9, y_test_9, y_val_9 = prepros_bancoimagenes(mydir_best_9, test_size, val_size)

x_train_10, x_test_10, x_val_10, y_train_10, y_test_10, y_val_10 = prepros_bancoimagenes(mydir_best_10, test_size, val_size)
x_train_11, x_test_11, x_val_11, y_train_11, y_test_11, y_val_11 = prepros_bancoimagenes(mydir_best_11, test_size, val_size)
x_train_12, x_test_12, x_val_12, y_train_12, y_test_12, y_val_12 = prepros_bancoimagenes(mydir_best_12, test_size, val_size)

(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)
(1086, 128, 128)
(1086,)


In [13]:
iteraciones = 10
iterar_cnn(iteraciones, x_train_1, y_train_1, x_val_1, y_val_1, 'r2-4096-GLOBAL', epochs, batch)
iterar_cnn(iteraciones, x_train_2, y_train_2, x_val_2, y_val_2, 'r2-4096-LOCAL', epochs, batch)
iterar_cnn(iteraciones, x_train_3, y_train_3, x_val_3, y_val_3, 'r2-4096-SENAL', epochs, batch)


In [14]:
iterar_cnn(iteraciones, x_train_4, y_train_4, x_val_4, y_val_4, 'r2-8192-GLOBAL', epochs, batch)
iterar_cnn(iteraciones, x_train_5, y_train_5, x_val_5, y_val_5, 'r2-8192-LOCAL', epochs, batch)
iterar_cnn(iteraciones, x_train_6, y_train_6, x_val_6, y_val_6, 'r2-8192-SENAL', epochs, batch)

In [15]:
iterar_cnn(iteraciones, x_train_7, y_train_7, x_val_7, y_val_7, 'r2-16384-GLOBAL', epochs, batch)
iterar_cnn(iteraciones, x_train_8, y_train_8, x_val_8, y_val_8, 'r2-16384-LOCAL', epochs, batch)
iterar_cnn(iteraciones, x_train_9, y_train_9, x_val_9, y_val_9, 'r2-16384-SENAL', epochs, batch)

In [13]:
iteraciones = 10
iterar_cnn(iteraciones, x_train_10, y_train_10, x_val_10, y_val_10, 'r2-32768-GLOBAL', epochs, batch)
iterar_cnn(iteraciones, x_train_11, y_train_11, x_val_11, y_val_11, 'r2-32768-LOCAL', epochs, batch)
iterar_cnn(iteraciones, x_train_12, y_train_12, x_val_12, y_val_12, 'r2-32768-SENAL', epochs, batch)

# Para resultados 2: Distintos anchos de ventana duración señal 0,6 seg.

In [14]:
#Directorios de bancos de imagenes


mydir_best_1= r"BANCOS/grey/4096/0,6seg/Banco_LOCAL_COLOR" 
mydir_best_2= r"BANCOS/grey/8192/0,6seg/Banco_LOCAL_COLOR" 
mydir_best_3 = r"BANCOS/grey/32768/0,6seg/Banco_LOCAL_COLOR"

test_size = 0.2
val_size = 0.1

In [15]:
#Datos
x_train_1, x_test_1, x_val_1, y_train_1, y_test_1, y_val_1 = prepros_bancoimagenes(mydir_best_1, test_size, val_size)
x_train_2, x_test_2, x_val_2, y_train_2, y_test_2, y_val_2 = prepros_bancoimagenes(mydir_best_2, test_size, val_size)
x_train_3, x_test_3, x_val_3, y_train_3, y_test_3, y_val_3 = prepros_bancoimagenes(mydir_best_3, test_size, val_size)

(944, 128, 128)
(944,)
(944, 128, 128)
(944,)
(944, 128, 128)
(944,)


In [18]:
iteraciones = 10
iterar_cnn(iteraciones, x_train_1, y_train_1, x_val_1, y_val_1, 'r2-4096-LOCAL-0,6seg', epochs, batch)
iterar_cnn(iteraciones, x_train_2, y_train_2, x_val_2, y_val_2, 'r2-8192-LOCAL-0,6seg', epochs, batch)
iterar_cnn(iteraciones, x_train_3, y_train_3, x_val_3, y_val_3, 'r2-32768-LOCAL-0,6seg', epochs, batch)


# Para resultados 2: Distintos anchos de ventana duración señal 1 seg.

In [19]:
#Directorios de bancos de imagenes


mydir_best_1= r"BANCOS/grey/4096/1seg/Banco_LOCAL_COLOR" 
mydir_best_2= r"BANCOS/grey/8192/1seg/Banco_LOCAL_COLOR" 
mydir_best_3 = r"BANCOS/grey/32768/1seg/Banco_LOCAL_COLOR"

test_size = 0.2
val_size = 0.1

In [20]:
#Datos
x_train_1, x_test_1, x_val_1, y_train_1, y_test_1, y_val_1 = prepros_bancoimagenes(mydir_best_1, test_size, val_size)
x_train_2, x_test_2, x_val_2, y_train_2, y_test_2, y_val_2 = prepros_bancoimagenes(mydir_best_2, test_size, val_size)
x_train_3, x_test_3, x_val_3, y_train_3, y_test_3, y_val_3 = prepros_bancoimagenes(mydir_best_3, test_size, val_size)

(590, 128, 128)
(590,)
(590, 128, 128)
(590,)
(590, 128, 128)
(590,)


In [21]:
iteraciones = 10
iterar_cnn(iteraciones, x_train_1, y_train_1, x_val_1, y_val_1, 'r2-4096-LOCAL-1seg', epochs, batch)
iterar_cnn(iteraciones, x_train_2, y_train_2, x_val_2, y_val_2, 'r2-8192-LOCAL-1seg', epochs, batch)
iterar_cnn(iteraciones, x_train_3, y_train_3, x_val_3, y_val_3, 'r2-32768-LOCAL-1seg', epochs, batch)
